In [11]:
from IPython.display import HTML

html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>CT Scan Contour Viewer</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/d3/7.8.5/d3.js"></script>
    <link rel="stylesheet" href="styles.css">
</head>

<body>
<div class="container">
    <h1>CT Scan Images</h1>

    <div class="slider">
        <label for="binCount">Bin Count</label>
        <input type="range" id="binCount" min="2" max="20" value="10" step="1">

        <label for="minThreshold">Min Threshold</label>
        <input type="range" id="minThreshold" min="0" max="100" value="0">

        <label for="maxThreshold">Max Threshold</label>
        <input type="range" id="maxThreshold" min="0" max="1000" value="300">
    </div>

    <hr />

    <div class="main">
        <div id="files_container">
            <select id="files" name="file" size="10" multiple="multiple"></select>
        </div>

        <div class="canvas">
            <svg viewBox="-5 -10 270 270"></svg>
        </div>
    </div>
</div>

<script>

const svg = d3.select("svg");
const path = d3.geoPath();

let currentBinCount = +document.querySelector("#binCount").value;
let currentMin = +document.querySelector("#minThreshold").value;
let currentMax = +document.querySelector("#maxThreshold").value;

function plot_contour(fileName) {

    const url =
        "https://raw.githubusercontent.com/umassdgithub/DataViz-Fall2025/main/Week-12-Part-2%20MultiView%20Tooltip/Activity%206/dicomData/" +
        fileName;

    d3.json(url).then(data => {

        svg.selectAll("*").remove();

        const m = 256, n = 256;

        // Flatten 2D array into 1D
        let values_density = [];
        data.forEach(col => col.forEach(d => values_density.push(+d)));

        const min_max = d3.extent(values_density);
        const minValue = min_max[0];
        const maxValue = min_max[1];

        // Update slider ranges
        document.querySelector("#minThreshold").min = minValue;
        document.querySelector("#minThreshold").max = maxValue;

        document.querySelector("#maxThreshold").min = minValue;
        document.querySelector("#maxThreshold").max = maxValue;

        function draw(binCount, minThreshold, maxThreshold) {

            svg.selectAll("*").remove();

            // ✅ Color scale
            const colors = d3.scaleLinear()
                .domain([
                    +minThreshold,
                    (+minThreshold + +maxThreshold) / 2,
                    +maxThreshold
                ])
                .range([
                    "#0d1a50",
                    "#3e5eba",
                    "#2b83ba",
                    "#abdda4",
                    "#ffffbf",
                    "#fdae61",
                    "#d7191c"
                ])
                .interpolate(d3.interpolateHcl);

            // ✅ Threshold step
            const step = (maxThreshold - minThreshold) / binCount;

            // ✅ Contour generation
            const contours = d3.contours()
                .size([m, n])
                .thresholds(d3.range(+minThreshold, +maxThreshold, step))
                (values_density);

            // ✅ Draw contours
            svg.append("g")
                .attr("class", "contours")
                .selectAll("path")
                .data(contours)
                .enter()
                .append("path")
                .attr("d", d => path(d))
                .attr("stroke", "black")
                .attr("stroke-width", ".1px")
                .attr("stroke-linejoin", "round")
                .attr("fill", d => colors(d.value));
        }

        // Initial draw
        draw(currentBinCount, currentMin, currentMax);

        // Slider updates
        document.addEventListener("input", e => {
            if (["binCount", "minThreshold", "maxThreshold"].includes(e.target.id)) {
                currentBinCount = +document.querySelector("#binCount").value;
                currentMin = +document.querySelector("#minThreshold").value;
                currentMax = +document.querySelector("#maxThreshold").value;

                draw(currentBinCount, currentMin, currentMax);
            }
        });

    }).catch(err => {
        console.error("Error loading file:", fileName, err);
    });
}

// ✅ Populate file list
function filesList() {

    const files_data = [
        "brain_001.dcm.json", "brain_002.dcm.json", "brain_003.dcm.json",
        "brain_004.dcm.json", "brain_005.dcm.json", "brain_006.dcm.json",
        "brain_007.dcm.json", "brain_008.dcm.json", "brain_009.dcm.json",
        "brain_010.dcm.json", "brain_011.dcm.json", "brain_012.dcm.json",
        "brain_013.dcm.json", "brain_014.dcm.json", "brain_015.dcm.json",
        "brain_016.dcm.json", "brain_017.dcm.json", "brain_018.dcm.json",
        "brain_019.dcm.json", "brain_020.dcm.json"
    ];

    const selectElement = d3.select("#files");

    selectElement.selectAll("option")
        .data(files_data)
        .enter()
        .append("option")
        .text(d => d);

    selectElement.on("change", () => {
        const selected = selectElement.property("value");
        plot_contour(selected);
    });
}

// ✅ Initialize
filesList();
plot_contour("brain_001.dcm.json");

</script>
</body>
</html>
"""

display(HTML(html_content))